#E-Learning Platform Database - EduHub MongoDB Project


##Database Setup and Data Modeling


In [ ]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import pprint

# Establish connection
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

# Create collections with validation rules
db.create_collection('users', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['userId', 'email', 'firstName', 'lastName', 'role', 'dateJoined'],
        'properties': {
            'userId': {'bsonType': 'string', 'description': 'must be a string and is required'},
            'email': {
                'bsonType': 'string',
                'pattern': '^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\\.[a-zA-Z]{2,}$',
                'description': 'must be a valid email and is required'
            },
            'firstName': {'bsonType': 'string', 'description': 'must be a string and is required'},
            'lastName': {'bsonType': 'string', 'description': 'must be a string and is required'},
            'role': {
                'enum': ['student', 'instructor', 'admin'],
                'description': 'must be one of student, instructor, or admin'
            },
            'dateJoined': {'bsonType': 'date', 'description': 'must be a date'},
            'profile': {
                'bsonType': 'object',
                'properties': {
                    'bio': {'bsonType': 'string'},
                    'avatar': {'bsonType': 'string'},
                    'skills': {
                        'bsonType': 'array',
                        'items': {'bsonType': 'string'}
                    }
                }
            },
            'isActive': {'bsonType': 'bool', 'description': 'must be a boolean'}
        }
    }
})

db.create_collection('courses', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['courseId', 'title', 'instructorId', 'createdAt'],
        'properties': {
            'courseId': {'bsonType': 'string', 'description': 'must be a string and is required'},
            'title': {'bsonType': 'string', 'description': 'must be a string and is required'},
            'description': {'bsonType': 'string'},
            'instructorId': {'bsonType': 'string', 'description': 'must reference a user'},
            'category': {'bsonType': 'string'},
            'level': {
                'enum': ['beginner', 'intermediate', 'advanced'],
                'description': 'must be one of beginner, intermediate, or advanced'
            },
            'duration': {'bsonType': 'number', 'minimum': 0},
            'price': {'bsonType': 'number', 'minimum': 0},
            'tags': {
                'bsonType': 'array',
                'items': {'bsonType': 'string'}
            },
            'createdAt': {'bsonType': 'date'},
            'updatedAt': {'bsonType': 'date'},
            'isPublished': {'bsonType': 'bool'}
        }
    }
})

db.create_collection('enrollments', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['enrollmentId', 'studentId', 'courseId', 'enrollmentDate'],
        'properties': {
            'enrollmentId': {'bsonType': 'string'},
            'studentId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},
            'enrollmentDate': {'bsonType': 'date'},
            'completionStatus': {
                'enum': ['not_started', 'in_progress', 'completed'],
                'description': 'must be one of not_started, in_progress, or completed'
            },
            'progress': {
                'bsonType': 'number',
                'minimum': 0,
                'maximum': 100
            },
            'lastAccessed': {'bsonType': 'date'}
        }
    }
})

db.create_collection('lessons', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['lessonId', 'courseId', 'title', 'sequence'],
        'properties': {
            'lessonId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},
            'title': {'bsonType': 'string'},
            'content': {'bsonType': 'string'},
            'sequence': {'bsonType': 'number', 'minimum': 1},
            'duration': {'bsonType': 'number', 'minimum': 0},
            'resources': {
                'bsonType': 'array',
                'items': {
                    'bsonType': 'object',
                    'properties': {
                        'type': {'bsonType': 'string'},
                        'url': {'bsonType': 'string'},
                        'description': {'bsonType': 'string'}
                    }
                }
            },
            'createdAt': {'bsonType': 'date'}
        }
    }
})

db.create_collection('assignments', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['assignmentId', 'courseId', 'title', 'dueDate'],
        'properties': {
            'assignmentId': {'bsonType': 'string'},
            'courseId': {'bsonType': 'string'},
            'title': {'bsonType': 'string'},
            'description': {'bsonType': 'string'},
            'dueDate': {'bsonType': 'date'},
            'maxScore': {'bsonType': 'number', 'minimum': 0},
            'createdAt': {'bsonType': 'date'}
        }
    }
})

db.create_collection('submissions', validator={
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ['submissionId', 'assignmentId', 'studentId', 'submissionDate'],
        'properties': {
            'submissionId': {'bsonType': 'string'},
            'assignmentId': {'bsonType': 'string'},
            'studentId': {'bsonType': 'string'},
            'submissionDate': {'bsonType': 'date'},
            'content': {'bsonType': 'string'},
            'grade': {
                'bsonType': 'number',
                'minimum': 0
            },
            'feedback': {'bsonType': 'string'},
            'isGraded': {'bsonType': 'bool'}
        }
    }
})

# Create indexes for performance optimization
db.users.create_index([('email', 1)], unique=True)
db.users.create_index([('userId', 1)], unique=True)
db.users.create_index([('role', 1)])

db.courses.create_index([('courseId', 1)], unique=True)
db.courses.create_index([('title', 'text'), ('description', 'text')])
db.courses.create_index([('instructorId', 1)])
db.courses.create_index([('category', 1)])
db.courses.create_index([('isPublished', 1)])

db.enrollments.create_index([('enrollmentId', 1)], unique=True)
db.enrollments.create_index([('studentId', 1), ('courseId', 1)], unique=True)
db.enrollments.create_index([('courseId', 1)])

db.lessons.create_index([('lessonId', 1)], unique=True)
db.lessons.create_index([('courseId', 1), ('sequence', 1)])

db.assignments.create_index([('assignmentId', 1)], unique=True)
db.assignments.create_index([('courseId', 1)])
db.assignments.create_index([('dueDate', 1)])

db.submissions.create_index([('submissionId', 1)], unique=True)
db.submissions.create_index([('assignmentId', 1), ('studentId', 1)])
db.submissions.create_index([('studentId', 1)])


##Data Population


In [ ]:
# Insert sample users
users_data = [
    {
        "userId": "user1",
        "email": "john.doe@example.com",
        "firstName": "John",
        "lastName": "Doe",
        "role": "student",
        "dateJoined": datetime(2024, 1, 15),
        "profile": {
            "bio": "Computer science student",
            "skills": ["Python", "Java"]
        },
        "isActive": True
    },
    {
        "userId": "user2",
        "email": "jane.smith@example.com",
        "firstName": "Jane",
        "lastName": "Smith",
        "role": "student",
        "dateJoined": datetime(2024, 2, 10),
        "profile": {
            "bio": "Data science enthusiast",
            "skills": ["Python", "R", "SQL"]
        },
        "isActive": True
    },
    {
        "userId": "user3",
        "email": "prof.johnson@example.com",
        "firstName": "Robert",
        "lastName": "Johnson",
        "role": "instructor",
        "dateJoined": datetime(2023, 11, 5),
        "profile": {
            "bio": "Senior software engineer with 10 years of experience",
            "skills": ["JavaScript", "Node.js", "React"]
        },
        "isActive": True
    },
    {
        "userId": "user4",
        "email": "dr.williams@example.com",
        "firstName": "Sarah",
        "lastName": "Williams",
        "role": "instructor",
        "dateJoined": datetime(2023, 10, 20),
        "profile": {
            "bio": "PhD in Data Science",
            "skills": ["Machine Learning", "Python", "TensorFlow"]
        },
        "isActive": True
    }
]

db.users.insert_many(users_data)

# Insert sample courses
courses_data = [
    {
        "courseId": "course1",
        "title": "Introduction to Python Programming",
        "description": "Learn Python from scratch",
        "instructorId": "user3",
        "category": "Programming",
        "level": "beginner",
        "duration": 20,
        "price": 99.99,
        "tags": ["python", "programming", "beginner"],
        "createdAt": datetime(2024, 1, 1),
        "updatedAt": datetime(2024, 1, 10),
        "isPublished": True
    },
    {
        "courseId": "course2",
        "title": "Advanced Data Science with Python",
        "description": "Master data science techniques",
        "instructorId": "user4",
        "category": "Data Science",
        "level": "advanced",
        "duration": 40,
        "price": 199.99,
        "tags": ["data science", "python", "machine learning"],
        "createdAt": datetime(2024, 1, 5),
        "updatedAt": datetime(2024, 1, 15),
        "isPublished": True
    },
    {
        "courseId": "course3",
        "title": "Web Development with JavaScript",
        "description": "Build modern web applications",
        "instructorId": "user3",
        "category": "Web Development",
        "level": "intermediate",
        "duration": 30,
        "price": 149.99,
        "tags": ["javascript", "web", "react"],
        "createdAt": datetime(2024, 2, 1),
        "updatedAt": datetime(2024, 2, 10),
        "isPublished": True
    }
]

db.courses.insert_many(courses_data)

# Insert sample enrollments
enrollments_data = [
    {
        "enrollmentId": "enroll1",
        "studentId": "user1",
        "courseId": "course1",
        "enrollmentDate": datetime(2024, 1, 20),
        "completionStatus": "in_progress",
        "progress": 45,
        "lastAccessed": datetime(2024, 6, 1)
    },
    {
        "enrollmentId": "enroll2",
        "studentId": "user1",
        "courseId": "course2",
        "enrollmentDate": datetime(2024, 2, 15),
        "completionStatus": "in_progress",
        "progress": 20,
        "lastAccessed": datetime(2024, 6, 5)
    },
    {
        "enrollmentId": "enroll3",
        "studentId": "user2",
        "courseId": "course1",
        "enrollmentDate": datetime(2024, 2, 1),
        "completionStatus": "completed",
        "progress": 100,
        "lastAccessed": datetime(2024, 5, 30)
    }
]

db.enrollments.insert_many(enrollments_data)

# Insert sample lessons
lessons_data = [
    {
        "lessonId": "lesson1",
        "courseId": "course1",
        "title": "Python Basics",
        "content": "Introduction to Python syntax and basic concepts",
        "sequence": 1,
        "duration": 2,
        "resources": [
            {
                "type": "video",
                "url": "https://example.com/videos/python-basics",
                "description": "Introductory video"
            }
        ],
        "createdAt": datetime(2024, 1, 2)
    },
    {
        "lessonId": "lesson2",
        "courseId": "course1",
        "title": "Control Structures",
        "content": "Loops and conditionals in Python",
        "sequence": 2,
        "duration": 3,
        "createdAt": datetime(2024, 1, 3)
    },
    {
        "lessonId": "lesson3",
        "courseId": "course2",
        "title": "Data Cleaning Techniques",
        "content": "How to clean and preprocess data",
        "sequence": 1,
        "duration": 4,
        "createdAt": datetime(2024, 1, 6)
    }
]

db.lessons.insert_many(lessons_data)

# Insert sample assignments
assignments_data = [
    {
        "assignmentId": "assign1",
        "courseId": "course1",
        "title": "Python Basics Exercise",
        "description": "Complete the coding exercises",
        "dueDate": datetime(2024, 6, 15),
        "maxScore": 100,
        "createdAt": datetime(2024, 1, 5)
    },
    {
        "assignmentId": "assign2",
        "courseId": "course1",
        "title": "Control Structures Project",
        "description": "Build a small program using loops and conditionals",
        "dueDate": datetime(2024, 6, 30),
        "maxScore": 100,
        "createdAt": datetime(2024, 1, 10)
    },
    {
        "assignmentId": "assign3",
        "courseId": "course2",
        "title": "Data Cleaning Challenge",
        "description": "Clean the provided dataset",
        "dueDate": datetime(2024, 7, 10),
        "maxScore": 100,
        "createdAt": datetime(2024, 1, 15)
    }
]

db.assignments.insert_many(assignments_data)

# Insert sample submissions
submissions_data = [
    {
        "submissionId": "sub1",
        "assignmentId": "assign1",
        "studentId": "user1",
        "submissionDate": datetime(2024, 6, 10),
        "content": "https://example.com/submissions/user1/assign1",
        "grade": 85,
        "feedback": "Good job! Some minor improvements needed.",
        "isGraded": True
    },
    {
        "submissionId": "sub2",
        "assignmentId": "assign1",
        "studentId": "user2",
        "submissionDate": datetime(2024, 6, 12),
        "content": "https://example.com/submissions/user2/assign1",
        "grade": 92,
        "feedback": "Excellent work!",
        "isGraded": True
    },
    {
        "submissionId": "sub3",
        "assignmentId": "assign3",
        "studentId": "user1",
        "submissionDate": datetime(2024, 6, 5),
        "content": "https://example.com/submissions/user1/assign3",
        "isGraded": False
    }
]

db.submissions.insert_many(submissions_data)

##Basic CRUD Operations


###Create Operations

In [ ]:
# Add a new student user
new_student = {
    "userId": "user5",
    "email": "mike.brown@example.com",
    "firstName": "Mike",
    "lastName": "Brown",
    "role": "student",
    "dateJoined": datetime.now(),
    "isActive": True
}
db.users.insert_one(new_student)

# Create a new course
new_course = {
    "courseId": "course4",
    "title": "Introduction to Machine Learning",
    "description": "Learn the fundamentals of ML",
    "instructorId": "user4",
    "category": "Data Science",
    "level": "intermediate",
    "duration": 35,
    "price": 179.99,
    "tags": ["machine learning", "python", "ai"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": False
}
db.courses.insert_one(new_course)

# Enroll a student in a course
new_enrollment = {
    "enrollmentId": "enroll4",
    "studentId": "user2",
    "courseId": "course3",
    "enrollmentDate": datetime.now(),
    "completionStatus": "not_started",
    "progress": 0,
    "lastAccessed": datetime.now()
}
db.enrollments.insert_one(new_enrollment)

# Add a new lesson to an existing course
new_lesson = {
    "lessonId": "lesson4",
    "courseId": "course1",
    "title": "Functions in Python",
    "content": "Learn how to define and use functions",
    "sequence": 3,
    "duration": 3,
    "createdAt": datetime.now()
}
db.lessons.insert_one(new_lesson)

###Read Operations


In [ ]:
# Find all active students
active_students = db.users.find({
    "role": "student",
    "isActive": True
})
print("Active Students:")
for student in active_students:
    print(f"- {student['firstName']} {student['lastName']}")

# Retrieve course details with instructor information
course_with_instructor = db.courses.aggregate([
    {
        "$match": {"courseId": "course1"}
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {
        "$unwind": "$instructor"
    },
    {
        "$project": {
            "title": 1,
            "description": 1,
            "category": 1,
            "instructorName": {"$concat": ["$instructor.firstName", " ", "$instructor.lastName"]}
        }
    }
])
print("\nCourse with Instructor:")
pprint.pprint(list(course_with_instructor))

# Get all courses in a specific category
programming_courses = db.courses.find({
    "category": "Programming",
    "isPublished": True
})
print("\nProgramming Courses:")
for course in programming_courses:
    print(f"- {course['title']}")

# Find students enrolled in a particular course
course_students = db.enrollments.aggregate([
    {
        "$match": {"courseId": "course1"}
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "studentId",
            "foreignField": "userId",
            "as": "student"
        }
    },
    {
        "$unwind": "$student"
    },
    {
        "$project": {
            "studentName": {"$concat": ["$student.firstName", " ", "$student.lastName"]},
            "progress": 1,
            "completionStatus": 1
        }
    }
])
print("\nStudents in Course 1:")
pprint.pprint(list(course_students))

# Search courses by title (case-insensitive, partial match)
search_term = "python"
python_courses = db.courses.find({
    "$or": [
        {"title": {"$regex": search_term, "$options": "i"}},
        {"description": {"$regex": search_term, "$options": "i"}}
    ]
})
print(f"\nCourses related to '{search_term}':")
for course in python_courses:
    print(f"- {course['title']}")

###Update Operations


In [ ]:
# Update a user's profile information
db.users.update_one(
    {"userId": "user1"},
    {
        "$set": {
            "profile.bio": "Computer science student specializing in AI",
            "profile.skills": ["Python", "Java", "Machine Learning"]
        }
    }
)

# Mark a course as published
db.courses.update_one(
    {"courseId": "course4"},
    {
        "$set": {
            "isPublished": True,
            "updatedAt": datetime.now()
        }
    }
)

# Update assignment grades
db.submissions.update_one(
    {"submissionId": "sub3"},
    {
        "$set": {
            "grade": 78,
            "feedback": "Good work, but could be more thorough in data cleaning steps.",
            "isGraded": True
        }
    }
)

# Add tags to an existing course
db.courses.update_one(
    {"courseId": "course3"},
    {
        "$addToSet": {
            "tags": {"$each": ["frontend", "backend"]}
        },
        "$set": {
            "updatedAt": datetime.now()
        }
    }
)

###Delete Operations


In [ ]:
# Remove a user (soft delete by setting isActive to false)
db.users.update_one(
    {"userId": "user5"},
    {"$set": {"isActive": False}}
)

# Delete an enrollment
db.enrollments.delete_one({"enrollmentId": "enroll4"})

# Remove a lesson from a course
db.lessons.delete_one({"lessonId": "lesson4"})

##Advanced Queries and Aggregation


###Complex Queries


In [ ]:
# Find courses with price between $50 and $200
mid_price_courses = db.courses.find({
    "price": {"$gte": 50, "$lte": 200},
    "isPublished": True
})
print("Courses between $50 and $200:")
for course in mid_price_courses:
    print(f"- {course['title']}: ${course['price']}")

# Get users who joined in the last 6 months
six_months_ago = datetime.now() - pd.DateOffset(months=6).to_pytimedelta()
recent_users = db.users.find({
    "dateJoined": {"$gte": six_months_ago}
})
print("\nUsers who joined in the last 6 months:")
for user in recent_users:
    print(f"- {user['firstName']} {user['lastName']} ({user['dateJoined'].strftime('%Y-%m-%d')})")

# Find courses that have specific tags using $in operator
tags_to_find = ["python", "machine learning"]
tagged_courses = db.courses.find({
    "tags": {"$in": tags_to_find}
})
print(f"\nCourses with tags {tags_to_find}:")
for course in tagged_courses:
    print(f"- {course['title']} (Tags: {', '.join(course['tags'])})")

# Retrieve assignments with due dates in the next week
next_week = datetime.now() + pd.DateOffset(days=7).to_pytimedelta()
upcoming_assignments = db.assignments.find({
    "dueDate": {"$gte": datetime.now(), "$lte": next_week}
})
print("\nAssignments due in the next week:")
for assignment in upcoming_assignments:
    print(f"- {assignment['title']} (Due: {assignment['dueDate'].strftime('%Y-%m-%d')})")

###Aggregation Pipeline


In [ ]:
# Course Enrollment Statistics
enrollment_stats = db.enrollments.aggregate([
    {
        "$group": {
            "_id": "$courseId",
            "totalEnrollments": {"$sum": 1},
            "avgProgress": {"$avg": "$progress"}
        }
    },
    {
        "$lookup": {
            "from": "courses",
            "localField": "_id",
            "foreignField": "courseId",
            "as": "course"
        }
    },
    {
        "$unwind": "$course"
    },
    {
        "$project": {
            "courseTitle": "$course.title",
            "totalEnrollments": 1,
            "avgProgress": {"$round": ["$avgProgress", 2]}
        }
    },
    {
        "$sort": {"totalEnrollments": -1}
    }
])
print("Course Enrollment Statistics:")
pprint.pprint(list(enrollment_stats))

# Student Performance Analysis
student_performance = db.submissions.aggregate([
    {
        "$match": {"isGraded": True}
    },
    {
        "$group": {
            "_id": "$studentId",
            "averageGrade": {"$avg": "$grade"},
            "submissionCount": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "_id",
            "foreignField": "userId",
            "as": "student"
        }
    },
    {
        "$unwind": "$student"
    },
    {
        "$project": {
            "studentName": {"$concat": ["$student.firstName", " ", "$student.lastName"]},
            "averageGrade": {"$round": ["$averageGrade", 2]},
            "submissionCount": 1
        }
    },
    {
        "$sort": {"averageGrade": -1}
    }
])
print("\nStudent Performance Analysis:")
pprint.pprint(list(student_performance))

# Instructor Analytics
instructor_analytics = db.courses.aggregate([
    {
        "$match": {"isPublished": True}
    },
    {
        "$lookup": {
            "from": "enrollments",
            "localField": "courseId",
            "foreignField": "courseId",
            "as": "enrollments"
        }
    },
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    {
        "$unwind": "$instructor"
    },
    {
        "$project": {
            "instructorName": {"$concat": ["$instructor.firstName", " ", "$instructor.lastName"]},
            "courseTitle": "$title",
            "studentCount": {"$size": "$enrollments"},
            "revenue": {"$multiply": ["$price", {"$size": "$enrollments"}]}
        }
    },
    {
        "$group": {
            "_id": "$instructorName",
            "totalStudents": {"$sum": "$studentCount"},
            "totalRevenue": {"$sum": "$revenue"},
            "courseCount": {"$sum": 1}
        }
    },
    {
        "$sort": {"totalRevenue": -1}
    }
])
print("\nInstructor Analytics:")
pprint.pprint(list(instructor_analytics))

# Advanced Analytics: Monthly enrollment trends
monthly_enrollments = db.enrollments.aggregate([
    {
        "$project": {
            "year": {"$year": "$enrollmentDate"},
            "month": {"$month": "$enrollmentDate"}
        }
    },
    {
        "$group": {
            "_id": {
                "year": "$year",
                "month": "$month"
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"_id.year": 1, "_id.month": 1}
    },
    {
        "$project": {
            "monthYear": {
                "$dateToString": {
                    "format": "%Y-%m",
                    "date": {
                        "$dateFromParts": {
                            "year": "$_id.year",
                            "month": "$_id.month"
                        }
                    }
                }
            },
            "count": 1
        }
    }
])
print("\nMonthly Enrollment Trends:")
pprint.pprint(list(monthly_enrollments))

##Indexing and Performance


In [ ]:
# Analyze query performance before optimization
query = {"courseId": "course1"}
print("Query explanation before optimization:")
pprint.pprint(db.enrollments.find(query).explain())

# Create index for the query
db.enrollments.create_index([("courseId", 1)])

# Analyze query performance after optimization
print("\nQuery explanation after optimization:")
pprint.pprint(db.enrollments.find(query).explain())

# Compare performance with timing
import time

# Without index
start_time = time.time()
for _ in range(1000):
    db.enrollments.find({"studentId": "user1"})
no_index_time = time.time() - start_time

# Create index
db.enrollments.create_index([("studentId", 1)])

# With index
start_time = time.time()
for _ in range(1000):
    db.enrollments.find({"studentId": "user1"})
with_index_time = time.time() - start_time

print(f"\nPerformance Comparison:")
print(f"Without index: {no_index_time:.4f} seconds for 1000 queries")
print(f"With index: {with_index_time:.4f} seconds for 1000 queries")
print(f"Improvement: {(no_index_time - with_index_time)/no_index_time * 100:.1f}% faster")

##Data Validation and Error Handling


In [ ]:
# Test schema validation
try:
    # This should fail because email is missing
    db.users.insert_one({
        "userId": "test1",
        "firstName": "Test",
        "lastName": "User",
        "role": "student",
        "dateJoined": datetime.now()
    })
except Exception as e:
    print("\nValidation Error (missing email):")
    print(e)

try:
    # This should fail because role is invalid
    db.users.insert_one({
        "userId": "test2",
        "email": "test2@example.com",
        "firstName": "Test",
        "lastName": "User",
        "role": "invalid_role",
        "dateJoined": datetime.now()
    })
except Exception as e:
    print("\nValidation Error (invalid role):")
    print(e)

try:
    # This should fail because price is negative
    db.courses.insert_one({
        "courseId": "test_course",
        "title": "Test Course",
        "instructorId": "user3",
        "createdAt": datetime.now(),
        "price": -10
    })
except Exception as e:
    print("\nValidation Error (negative price):")
    print(e)

# Test duplicate key error
try:
    # This should fail because userId must be unique
    db.users.insert_one({
        "userId": "user1",
        "email": "new@example.com",
        "firstName": "New",
        "lastName": "User",
        "role": "student",
        "dateJoined": datetime.now()
    })
except Exception as e:
    print("\nDuplicate Key Error:")
    print(e)